In [1]:
import keras
from keras.layers import LSTM,Dense,Input,Flatten,Concatenate
from keras.models import Model,load_model
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot

Using TensorFlow backend.


In [178]:
def build_model(n_units):
    x_in = Input(shape=(1,9))
    #lstm_out,lstm_h,lstm_c = LSTM(n_units, return_state = True,stateful=True)(x_in)
    
    h_in = Input(shape=(n_units,))
    c_in = Input(shape=(n_units,))
    lstm = LSTM(n_units, return_state = True)
    lstm_out,lstm_h,lstm_c = lstm(x_in,initial_state =[h_in,c_in])
    
    #x = Flatten()(x_in)
    #lstm_con = Concatenate()([lstm_h,lstm_c])
    
    y_out = Dense(1,activation = 'relu')(lstm_out)
    #y_out = Dense(1,activation = 'relu')(y)
    return Model(inputs=[x_in,h_in,c_in],outputs=[y_out,lstm_h,lstm_c])

In [2]:
def build_model(n_units):
    x_in = Input(batch_shape=(1,1,9))
    lstm_out,lstm_h,lstm_c = LSTM(n_units, return_state = True,stateful=True)(x_in)
    '''
    h_in = Input(shape=(n_units,))
    c_in = Input(shape=(n_units,))
    lstm = LSTM(n_units, return_state = True)
    lstm_out,lstm_h,lstm_c = lstm(x_in,initial_state =[h_in,c_in])
    '''
    #x = Flatten()(x_in)
    #lstm_con = Concatenate()([lstm_h,lstm_c])
    
    y_out = Dense(1,activation = 'relu')(lstm_out)
    #y_out = Dense(1,activation = 'relu')(y)
    return Model(inputs=x_in,outputs=y_out)

In [3]:
b_size = 1
realtime_LSTM= build_model(128)
opt = keras.optimizers.Adam(lr=0.000001)
realtime_LSTM.compile(optimizer =opt, loss='mse',metrics=['acc'])
#realtime_LSTM.load_weights('realtime_LSTM_useOutepoch_5000loss_0.01_weights.h5')

In [4]:
realtime_LSTM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (1, 1, 9)                 0         
_________________________________________________________________
lstm_1 (LSTM)                [(1, 128), (1, 128), (1,  70656     
_________________________________________________________________
dense_1 (Dense)              (1, 1)                    129       
Total params: 70,785
Trainable params: 70,785
Non-trainable params: 0
_________________________________________________________________


In [20]:
x=np.load('PSDSData (4)_bf7ff952e4163201_raw.npy')

print(x.shape)
x=x.reshape(len(x),1,9)

(7610, 9)


In [193]:
y=np.load('watch_4_29_y.npy')
#y=y.reshape((y.shape[0],1))
y.shape

(20941,)

In [194]:
y=np.append(np.zeros(31),y)

In [11]:
realtime_LSTM.load_weights('realtime_LSTM_v2_128Unitsepoch_950loss_0.16_weights.h5')

In [21]:
y_pre=realtime_LSTM.predict(x,batch_size=1)

In [8]:
weights= realtime_LSTM.get_weights()

In [9]:
for weight in weights:
    print(weight.shape)

(9, 512)
(128, 512)
(512,)
(128, 1)
(1,)


In [205]:
y_pre.shape

(20972, 1)

In [22]:
np.savetxt('realTimeY_128Units_xing&Guo.csv',y_pre)

In [196]:
sample_weight=y*15+1

In [201]:
realtime_LSTM.fit(x,y,batch_size=1,sample_weight=sample_weight,shuffle=False,epochs=50)

Epoch 1/50
20972/20972 [==============================] - 120s 6ms/step - loss: 0.1860 - acc: 0.8595
Epoch 2/50
20972/20972 [==============================] - 115s 5ms/step - loss: 0.1841 - acc: 0.8595
Epoch 3/50
20972/20972 [==============================] - 119s 6ms/step - loss: 0.1821 - acc: 0.8598
Epoch 4/50
20972/20972 [==============================] - 124s 6ms/step - loss: 0.1801 - acc: 0.8602
Epoch 5/50
20972/20972 [==============================] - 122s 6ms/step - loss: 0.1784 - acc: 0.8601
Epoch 6/50
20972/20972 [==============================] - 120s 6ms/step - loss: 0.1773 - acc: 0.8604
Epoch 7/50
20972/20972 [==============================] - 147s 7ms/step - loss: 0.1765 - acc: 0.8604
Epoch 8/50
20972/20972 [==============================] - 133s 6ms/step - loss: 0.1759 - acc: 0.8602
Epoch 9/50
20972/20972 [==============================] - 113s 5ms/step - loss: 0.1753 - acc: 0.8603
Epoch 10/50
20972/20972 [==============================] - 114s 5ms/step - loss: 0.1747 - a

In [185]:
y=[]
h=np.zeros((1,256))
c=np.zeros((1,256))
for i in range(len(x)):
    x_in = x[i].reshape((1,1,9))
    y_out,h,c=realtime_LSTM.predict([x_in,h,c],batch_size=1)
    y.append(y_out.reshape((1)))


In [186]:
y=np.asarray(y)
y.shape
np.savetxt('realTimeY.csv',y)

In [187]:
y.shape

(20972, 1)

In [73]:
y=np.load('watch_4_29_y.npy')
#y=y.reshape((y.shape[0],1))
y.shape


(20941,)

In [74]:
sample_weight=y*15+1

In [80]:
realtime_LSTM.fit(x[:,:,0:9],y,epochs=100,batch_size=64,sample_weight=sample_weight)


Epoch 1/100
20941/20941 [==============================] - 22s 1ms/step - loss: 0.4945 - acc: 0.5693
Epoch 2/100
20941/20941 [==============================] - 25s 1ms/step - loss: 0.3620 - acc: 0.6750
Epoch 3/100
20941/20941 [==============================] - 25s 1ms/step - loss: 0.3027 - acc: 0.7354
Epoch 4/100
20941/20941 [==============================] - 27s 1ms/step - loss: 0.2690 - acc: 0.7750
Epoch 5/100
20941/20941 [==============================] - 25s 1ms/step - loss: 0.2469 - acc: 0.8014
Epoch 6/100
20941/20941 [==============================] - 21s 1ms/step - loss: 0.2317 - acc: 0.8160
Epoch 7/100
20941/20941 [==============================] - 21s 1ms/step - loss: 0.2207 - acc: 0.8267
Epoch 8/100
20941/20941 [==============================] - 24s 1ms/step - loss: 0.2122 - acc: 0.8394
Epoch 9/100
20941/20941 [==============================] - 22s 1ms/step - loss: 0.2056 - acc: 0.8469
Epoch 10/100
20941/20941 [==============================] - 21s 982us/step - loss: 0.2003 -

In [82]:
lstm_auto_encoder=load_model('lstm_auto_encoder.h5')

C:\Users\Guo\Anaconda3\envs\Tensorflow_cpu\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [83]:
lstm_auto_encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 18)            0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 281600    
Total params: 281,600
Trainable params: 281,600
Non-trainable params: 0
_________________________________________________________________


In [91]:
def build_encoder(encoder):
    for l in encoder.layers:
        l.trainable=False
    x_in = Input(shape=(32,18))
    lstm_h,lstm_c = encoder((x_in))
    lstm_con = Concatenate()([lstm_h,lstm_c])
    
    y = Dense(32,activation = 'relu')(lstm_con)
    y_out = Dense(1,activation = 'relu')(y)
    return Model(inputs=x_in,outputs=y_out)

In [102]:
encoderModel=build_encoder(lstm_auto_encoder)
opt = keras.optimizers.Adam(lr=0.00001)
encoderModel.compile(optimizer =opt, loss='mse',metrics=['acc'])

In [103]:
encoderModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 32, 18)       0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 256), (None, 281600      input_25[0][0]                   
__________________________________________________________________________________________________
concatenate_10 (Concatenate)    (None, 512)          0           model_1[6][0]                    
                                                                 model_1[6][1]                    
__________________________________________________________________________________________________
dense_37 (Dense)                (None, 32)           16416       concatenate_10[0][0]             
__________

In [106]:
encoderModel.fit(x,y,epochs=1000,batch_size=64,sample_weight=sample_weight)

Epoch 1/1000
20941/20941 [==============================] - 6s 289us/step - loss: 0.1064 - acc: 0.9081
Epoch 2/1000
20941/20941 [==============================] - 6s 307us/step - loss: 0.1061 - acc: 0.9080
Epoch 3/1000
20941/20941 [==============================] - 7s 330us/step - loss: 0.1058 - acc: 0.9078
Epoch 4/1000
20941/20941 [==============================] - 6s 302us/step - loss: 0.1058 - acc: 0.9078
Epoch 5/1000
20941/20941 [==============================] - 6s 307us/step - loss: 0.1055 - acc: 0.9077
Epoch 6/1000
20941/20941 [==============================] - 6s 293us/step - loss: 0.1053 - acc: 0.9089
Epoch 7/1000
20941/20941 [==============================] - 6s 294us/step - loss: 0.1050 - acc: 0.9082
Epoch 8/1000
20941/20941 [==============================] - 7s 318us/step - loss: 0.1051 - acc: 0.9084
Epoch 9/1000
20941/20941 [==============================] - 6s 304us/step - loss: 0.1046 - acc: 0.9086
Epoch 10/1000
20941/20941 [==============================] - 7s 311us/ste

Epoch 79/1000
20941/20941 [==============================] - 5s 257us/step - loss: 0.0933 - acc: 0.9150
Epoch 80/1000
20941/20941 [==============================] - 5s 258us/step - loss: 0.0932 - acc: 0.9163
Epoch 81/1000
20941/20941 [==============================] - 6s 283us/step - loss: 0.0930 - acc: 0.9160
Epoch 82/1000
20941/20941 [==============================] - 6s 269us/step - loss: 0.0927 - acc: 0.9158
Epoch 83/1000
20941/20941 [==============================] - 6s 268us/step - loss: 0.0927 - acc: 0.9163
Epoch 84/1000
20941/20941 [==============================] - 6s 278us/step - loss: 0.0924 - acc: 0.9159
Epoch 85/1000
20941/20941 [==============================] - 6s 270us/step - loss: 0.0925 - acc: 0.9163
Epoch 86/1000
20941/20941 [==============================] - 5s 261us/step - loss: 0.0922 - acc: 0.9154
Epoch 87/1000
20941/20941 [==============================] - 6s 294us/step - loss: 0.0923 - acc: 0.9171
Epoch 88/1000
20941/20941 [==============================] - 6s 

20941/20941 [==============================] - 6s 269us/step - loss: 0.0764 - acc: 0.9294
Epoch 233/1000
20941/20941 [==============================] - 6s 276us/step - loss: 0.0764 - acc: 0.9289
Epoch 234/1000
20941/20941 [==============================] - 7s 315us/step - loss: 0.0762 - acc: 0.9286
Epoch 235/1000
20941/20941 [==============================] - 7s 322us/step - loss: 0.0764 - acc: 0.9288
Epoch 236/1000
20941/20941 [==============================] - 6s 303us/step - loss: 0.0762 - acc: 0.9289
Epoch 237/1000
20941/20941 [==============================] - 6s 288us/step - loss: 0.0761 - acc: 0.9293
Epoch 238/1000
20941/20941 [==============================] - 7s 320us/step - loss: 0.0760 - acc: 0.9295
Epoch 239/1000
20941/20941 [==============================] - 6s 293us/step - loss: 0.0760 - acc: 0.9286
Epoch 240/1000
20941/20941 [==============================] - 6s 309us/step - loss: 0.0761 - acc: 0.9294
Epoch 241/1000
20941/20941 [==============================] - 7s 330us

20941/20941 [==============================] - 6s 268us/step - loss: 0.0653 - acc: 0.9400
Epoch 386/1000
20941/20941 [==============================] - 6s 269us/step - loss: 0.0655 - acc: 0.9407
Epoch 387/1000
20941/20941 [==============================] - 6s 293us/step - loss: 0.0651 - acc: 0.9404
Epoch 388/1000
20941/20941 [==============================] - 6s 270us/step - loss: 0.0653 - acc: 0.9399
Epoch 389/1000
20941/20941 [==============================] - 6s 267us/step - loss: 0.0652 - acc: 0.9403
Epoch 390/1000
20941/20941 [==============================] - 6s 281us/step - loss: 0.0651 - acc: 0.9403
Epoch 391/1000
20941/20941 [==============================] - 6s 264us/step - loss: 0.0651 - acc: 0.9405
Epoch 392/1000
20941/20941 [==============================] - 5s 262us/step - loss: 0.0650 - acc: 0.9404
Epoch 393/1000
20941/20941 [==============================] - 6s 279us/step - loss: 0.0650 - acc: 0.9405
Epoch 394/1000
20941/20941 [==============================] - 6s 271us

20941/20941 [==============================] - 6s 276us/step - loss: 0.0570 - acc: 0.9489
Epoch 538/1000
20941/20941 [==============================] - 5s 258us/step - loss: 0.0571 - acc: 0.9483
Epoch 539/1000
20941/20941 [==============================] - 6s 281us/step - loss: 0.0569 - acc: 0.9481 4s - loss: - ETA: 3s - lo
Epoch 540/1000
20941/20941 [==============================] - 6s 267us/step - loss: 0.0569 - acc: 0.9486
Epoch 541/1000
20941/20941 [==============================] - 5s 260us/step - loss: 0.0569 - acc: 0.9487
Epoch 542/1000
20941/20941 [==============================] - 6s 278us/step - loss: 0.0568 - acc: 0.9486
Epoch 543/1000
20941/20941 [==============================] - 5s 260us/step - loss: 0.0567 - acc: 0.9491
Epoch 544/1000
20941/20941 [==============================] - 5s 261us/step - loss: 0.0568 - acc: 0.9484
Epoch 545/1000
20941/20941 [==============================] - 6s 278us/step - loss: 0.0568 - acc: 0.9487 3s - loss
Epoch 546/1000
20941/20941 [======

20941/20941 [==============================] - 5s 258us/step - loss: 0.0536 - acc: 0.9522
Epoch 614/1000
20941/20941 [==============================] - 6s 274us/step - loss: 0.0536 - acc: 0.9518
Epoch 615/1000
20941/20941 [==============================] - 5s 259us/step - loss: 0.0534 - acc: 0.9522
Epoch 616/1000
20941/20941 [==============================] - 5s 260us/step - loss: 0.0534 - acc: 0.9520
Epoch 617/1000
20941/20941 [==============================] - 6s 277us/step - loss: 0.0533 - acc: 0.9521
Epoch 618/1000
20941/20941 [==============================] - 5s 258us/step - loss: 0.0534 - acc: 0.9528
Epoch 619/1000
20941/20941 [==============================] - 6s 265us/step - loss: 0.0533 - acc: 0.9521
Epoch 620/1000
20941/20941 [==============================] - 6s 275us/step - loss: 0.0534 - acc: 0.9527
Epoch 621/1000
20941/20941 [==============================] - 5s 261us/step - loss: 0.0532 - acc: 0.9522
Epoch 622/1000
20941/20941 [==============================] - 6s 273us

20941/20941 [==============================] - 6s 282us/step - loss: 0.0475 - acc: 0.9583
Epoch 764/1000
20941/20941 [==============================] - 6s 299us/step - loss: 0.0475 - acc: 0.9585
Epoch 765/1000
20941/20941 [==============================] - 6s 284us/step - loss: 0.0475 - acc: 0.9575
Epoch 766/1000
20941/20941 [==============================] - 6s 307us/step - loss: 0.0475 - acc: 0.9585
Epoch 767/1000
20941/20941 [==============================] - 6s 296us/step - loss: 0.0474 - acc: 0.9585
Epoch 768/1000
20941/20941 [==============================] - 6s 294us/step - loss: 0.0473 - acc: 0.9585
Epoch 769/1000
20941/20941 [==============================] - 6s 303us/step - loss: 0.0473 - acc: 0.9586
Epoch 770/1000
20941/20941 [==============================] - 6s 296us/step - loss: 0.0473 - acc: 0.9588 1s - lo
Epoch 771/1000
20941/20941 [==============================] - 6s 307us/step - loss: 0.0472 - acc: 0.9586
Epoch 772/1000
20941/20941 [==============================] - 

20941/20941 [==============================] - 6s 282us/step - loss: 0.0423 - acc: 0.9632
Epoch 916/1000
20941/20941 [==============================] - 5s 262us/step - loss: 0.0422 - acc: 0.9631
Epoch 917/1000
20941/20941 [==============================] - 5s 261us/step - loss: 0.0423 - acc: 0.9634
Epoch 918/1000
20941/20941 [==============================] - 6s 279us/step - loss: 0.0423 - acc: 0.9633
Epoch 919/1000
20941/20941 [==============================] - 6s 269us/step - loss: 0.0421 - acc: 0.9636
Epoch 920/1000
20941/20941 [==============================] - 6s 275us/step - loss: 0.0419 - acc: 0.9632 0s - loss: 0.0421 - acc: 0.963
Epoch 921/1000
20941/20941 [==============================] - 6s 275us/step - loss: 0.0420 - acc: 0.9636
Epoch 922/1000
20941/20941 [==============================] - 6s 264us/step - loss: 0.0421 - acc: 0.9632
Epoch 923/1000
20941/20941 [==============================] - 6s 278us/step - loss: 0.0419 - acc: 0.9632
Epoch 924/1000
20941/20941 [===========

In [108]:
realTimeY=encoderModel.predict(x)

In [111]:
realTimeY=np.append(np.zeros(31),realTimeY)
np.savetxt('realTimeY.csv',realTimeY)